# FLDAS Explorer Dashboard
Modified: Jun 17, 2019

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
#import utils
import hv_utils as  hvu

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
fpath_sa = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
fpath_ea = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

In [ ]:
%%opts WMTS [width=W, height=H]
# basemap * timg_ea
# gvts.EsriNatGeo*timg_ea

---
## Add Callbacks 

### LatLon Tab selector stream

Fetch the lat,lon location from the mouse location

In [ ]:
from holoviews.streams import Tap, Selection1D, PointerXY, RangeXY

In [ ]:
# Defne time sereis scatter point selector stream
# tscatter_selection = Selection1D(source=tseries.Scatter.I)

In [ ]:
################################
from ipywidgets import Output
out = Output()

@out.capture(clear_output=True)
def listener(*args, **kwargs):
    print('Scatter selector listener called')
    print(args)
    print(kwargs)
################################
# tscatter_selection = Selection1D(source=tseries.Scatter.I)
# tscatter_selection.add_subscriber(listener)
out
################################

In [ ]:
def cb_tscatter(xrd, varname, index):
    print('cb_tscatter called. Selected indices: ', index)
    if not index:
        index = [0] #todo: current tindex
    tidx = index[0]
    return hvu.get_img(xrd, varname, tidx)

In [ ]:
# tseries_opts = {
    
def cb_tseries(xrd, varname, x, y, method='nearest'):
    print(f'x,y: {x,y}')
    tseries = xrd[varname].sel(X=x, Y=y,method=method)
    tseries_label = f"Time Series at Lon,Lat = ({x:.2f},{y:.2f}) "
    
    # Time series as Scatter and Curve
    scatter = hv.Scatter(tseries)
    curve = hv.Curve(tseries)
    
    # Link a stream to the scatter to enable Tab selection
    scatter_stream = Selection1D(source=scatter)
    scatter_stream.add_subscriber(listener) # for debug
    dmap_scatter_stream = hv.DynamicMap(
        lambda index: cb_tscatter(xrd, varname, index),
        streams=[scatter_stream])
    
    # Add HLine at mean over time
    mean = hvu.extract_item(tseries.mean())
    print(mean)
    mean_line = hv.HLine(mean, label='tseries_mean')
    t_midpoint = pd.Timestamp(hvu.extract_item(tseries.coords['time'][len(tseries)//2]))
    mean_label = hv.Labels([(t_midpoint, mean, f'mean: {mean:.3f}')])
    
    # Put together to an Overlay
    overlay = (
        curve.opts(alpha=0.5, line_width=1) 
        * scatter.opts(width=W, padding=0.2, size=5) 
        * mean_line.opts(color='black', alpha=0.5, line_width=1)
        * mean_label.opts(text_font_size='8pt',text_alpha=0.5)
    ) + dmap_scatter_stream
    return overlay.relabel(tseries_label)


    

---
### Add another callback for time series statistics
- On LatLon selection, compute the statistics of the current `varname` at `latlon_selected` across time.


In [ ]:
def cb_tstats(xrd, varname, x, y, 
              method='nearest',
             show_as_timestamp=True,
             decimals=3):
    tseries = xrd[varname].sel(X=x, Y=y,method=method)
    df = hvu.get_stats(tseries, 
                       show_as_timestamp=show_as_timestamp,
                       decimals=decimals)
    
    # Add metadata on selected latlon point
#     df['point_idx'] = index[0]#
    df['lat'] = y
    df['lon'] = x
    
    cols = df.columns.to_list()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]
    label = f"Time Series Stats at Lon,Lat = ({x:.2f},{y:.2f}) "
    return hv.Table(df, label=label)

---
## Combine the two callbacks for LatLon Tab selector


In [ ]:
def cb_latlon_tab(xrd, varname, x, y, **kwargs):
    """
    Creates a hv.Overlay object with two elements
    - layout = tseries + table
    where 
        - tseries: itself an overlay with scatter, curve, hline and a text for 
        the time series data at the selected LonLat location
        - table: contains basic statistics of the time series data at the selected location
    Args:
    - xrd (xarray.Dataset)
    - varname (str)
    - x (int): index to xrd's 'X' dim
    - y (int): index to xrd's 'Y' dim
    
    kwargs:
    - method (str): xarray's .sel method. Default: 'nearest'
    - show_as_teimstamp (bool): if True, the table will show time data as time rather than an index
    - decimals (int): number of decimals to kwarg to np.around() for float point display
    
    Returns
    - layout (hv.Overlay)
    """
    # Get the kwarg values
    method = kwargs.get('method', 'nearest')
    show_as_timestamp = kwargs.get('show_as_timestamp', True)
    decimals = kwargs.get('decimals', 3)
    
    # Get hv elements
    tseries = cb_tseries(xrd, varname, x, y, method=method)
    tstats = cb_tstats(xrd, varname, x, y, 
                       method=method, show_as_timestamp=show_as_timestamp, decimals=decimals)
    return (tseries+ tstats)


---
## Add a callback for RangeXY stream
- Fetch the x and y ranges of the current view
    - todo: Fetch appropriate vector tile

In [ ]:
def cb_rangexy(x_range, y_range):
    lbrt = hvu.ranges2lbrt(x_range, y_range)
    print(f'x_range: {x_range}')
    print(f'y_range: {y_range}')
    print(f'lbrt: ', lbrt)
    
    df = pd.DataFrame( [lbrt], columns='min_x min_y max_x max_y'.split() )
    return hv.Table( df)


---
## Putting the streams together


## FLDASExplorer with panel
Modified: Jun 13, 2019

In [ ]:
import datetime as dt

class FLDASExplorer(param.Parameterized):
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[0],objects=varnames)
    time = param.Date(dt.datetime(2019,4,1), bounds=(dt.datetime(2019, 4, 1), dt.datetime(2019, 4, 30)))
    alpha = param.Number(default=1.0, bounds=(0.0,1.0))
#     cmap = param.ObjectSelector(default='fire', objects=['fire'])

                                                    
    @param.depends('region', 'varname', 'time')#, 'alpha')#, 'cmap')
    def view(self):
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = gv.Image(xrd.sel(time=self.time)[self.varname], ['X','Y'], crs=ccrs.PlateCarree())    
        
    #     datashade returns a hv.DynamicMap which dynamically re-renders this img as we zoom/pan
        return basemap*datashade(img.opts(**img_opts), 
                                 cmap=fire,#self.cmap, 
#                                  alpha=self.alpha,
                                 **dopts)


In [ ]:
explorer = FLDASExplorer()
# img_dmap = hv.DynamicMap(explorer.view)
app = pn.Row( explorer.param, explorer.view)

In [ ]:
app.servable()

## Before panel...
- currently no `datashade` applied to the img(ie.`timg_ea`)

In [ ]:
# Set extra style opts (in addition to default from above)
scatter_opts = dict(width=W, height=H,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W, height=H,
                  framewise=True)
img_opts = dict(width=W, height=H,
                axiswise=True, 
                framewise=False,
               )
tbl_opts = dict(width = W)

In [ ]:
# These will be panel params
varname = varnames[1]
xrd = xrd_ea
tidx = 10
# Get timg given xrd (region), varname, time point
timg = hvu.get_img(xrd, varname, tidx)

In [ ]:
# Define streams
## On main image: LatLon Tab stream
tap_latlon = Tap(name='tap_latlon', x = 0.0, y=0.0, source=timg)
dmap_tseries = hv.DynamicMap(
    lambda x,y, **kwargs: cb_latlon_tab(xrd, varname, x,y, **kwargs),
    streams=[tap_latlon]
)

## Range stream and dmap
# range_src = timg_ea
# x_range, y_range = hvu.lbrt2ranges(range_src.bounds.lbrt())
# rangexy = RangeXY(x_range = x_range,
#                   y_range = y_range,
#                   source=timg_ea)
# dmap_range_tbl = hv.DynamicMap(cb_rangexy, streams=[rangexy])

## Putting all together
layout = (basemap * timg_ea) + dmap_tseries.collate()
layout.opts(
    opts.Image(**img_opts),
    opts.Scatter(**scatter_opts),
    opts.Curve(**curve_opts),
    opts.Table(**tbl_opts) 
).cols(1)
  

## FLDASExplorer with panel


In [ ]:
# datashader opts
dopts = dict(width=W, height=H,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

class FLDASExplorer(param.Parameterized):
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[0],objects=varnames)
    time = param.Date(dt.datetime(2019,4,1), bounds=(dt.datetime(2019, 4, 1), dt.datetime(2019, 4, 30)))
    alpha = param.Number(default=1.0, bounds=(0.0,1.0))
#     cmap = param.ObjectSelector(default='fire', objects=['fire'])

                                                    
    @param.depends('region', 'varname', 'time')#, 'alpha')#, 'cmap')
    def view(self):
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = gv.Image(xrd.sel(time=self.time)[self.varname], ['X','Y'], crs=ccrs.PlateCarree())    
        
    #     datashade returns a hv.DynamicMap which dynamically re-renders this img as we zoom/pan
        return basemap*datashade(img.opts(**img_opts), 
                                 cmap=fire,#self.cmap, 
#                                  alpha=self.alpha,
                                 **dopts)


In [ ]:
explorer = FLDASExplorer()
# img_dmap = hv.DynamicMap(explorer.view)
app = pn.Row( explorer.param, explorer.view)

In [ ]:
app.servable()